In [1]:
# The following file is being used to test the Pipeline
import matplotlib.pyplot as plt
''' Import statements for all the files involved'''
from main import *
from utils import *
from wfield_utils import *
from registration import *
from hemocorrection import *
from denoising import *
from debug_visualize import *
from data_loading_functions import *
from data_preprocessor import *
from tqdm import tqdm

In [2]:
# The folllowing code is from main(). 
# params are global variables used to modify the code
params = {
    'path_to_session_data':       r"C:\Users\evilm\Downloads\07-18-2024",
    'pipe_num':           0,              # select desired pipeline number
    'get_raw_brightness': False,
    'get_DeltaF':         False,
    'get_zScore':         False
          }
# This is inside def main()
# For purposes of testing to visualise the data we will do analysis here.
#session_file_paths = get_file_paths(params['path_to_session_data'])
# create an obj for each recording within the session
# recordings = {}
# if session_file_paths != 0:
#     n = 1
#     recordings[f'recording_{n}'] = rawDataPreprocessor(params['path_to_session_data'])

# print(f'{datetime.datetime.now().time()}: Done loading session data into object')


In [ ]:
# Testing motion correction
#print(f'{datetime.datetime.now().time()}: Starting Motion Correction')
#_, _, motion_corrected_frames_all = motion_correct(dat=recordings['recording_1'].all_frames, out=None, mode='ecc', apply_shifts=True)
#print(f'{datetime.datetime.now().time()}: Done Motion Correction')
#np.save(params['path_to_session_data']+"\motion_corrected_data",motion_corrected_frames_all)

In [3]:
motion_corrected_frames_all = np.load(r"C:\Users\evilm\Downloads\07-18-2024\motion_corrected_data.npy")

In [4]:
blue_frames = motion_corrected_frames_all[0:2000,0,...]
violet_frames = motion_corrected_frames_all[0:2000,1,...]
motion_corrected_frames_all = []

# Testing Normalising 
blue_normalised_frames = [(frame-np.min(frame))/(np.max(frame)-np.min(frame)) for frame in tqdm(blue_frames[:])]
viol_normalised_frames = [(frame-np.min(frame))/(np.max(frame)-np.min(frame)) for frame in tqdm(violet_frames[:])]

hemo_corrected_frames = [(blue_normalised_frames[i] - viol_normalised_frames[i]) for i in tqdm(range(len(blue_normalised_frames)))]

100%|██████████| 2000/2000 [00:35<00:00, 56.20it/s] 


In [6]:
# %matplotlib inline
# import matplotlib.animation as animation
# plt.figure
# plt.imshow(blue_normalised_frames[0],cmap='gray')
# plt.show()

# plt.figure
# plt.imshow(viol_normalised_frames[0],cmap='gray')
# plt.show()

# hemo_corr = blue_normalised_frames[0]-viol_normalised_frames[0]
# plt.figure
# plt.imshow(hemo_corr,cmap='gray')
# plt.show()

# print(np.max(hemo_corr))
# print(np.min(hemo_corr))
# hemo_corr = (hemo_corr - np.min(hemo_corr))/(np.max(hemo_corr)-np.min(hemo_corr))

# plt.figure
# plt.imshow(hemo_corr,cmap='gray')
# plt.show()
# print(np.max(hemo_corr))
# print(np.min(hemo_corr))

frames = []
fig = plt.figure()
for i in tqdm(range(len(hemo_corrected_frames))):
    frames.append([plt.imshow(hemo_corrected_frames[i],
                             cmap=plt.cm.Greys_r, animated = True)])
ani = animation.ArtistAnimation(fig,frames,interval=40, blit = True, repeat_delay=1000)
#ni.save('movie.mp4')
plt.show()


100%|██████████| 2000/2000 [00:03<00:00, 525.51it/s]


In [ ]:
# Testing Denoising and Compression
motion_corrected_frames_all = np.load(r"C:\Users\evilm\Downloads\07-18-2024\motion_corrected_data.npy")
# 2. Denoising and Compression
print(f'{datetime.datetime.now().time()}: Starting  Blue Frames')
denoised_blue_frames, blueU, blueS, blueVT = denoise_svd(motion_corrected_frames_all[:,0,...],rank=200) # only denoise blue channel for now
print(f'{datetime.datetime.now().time()}: Done Denoising Blue Frames')

print(f'{datetime.datetime.now().time()}: Starting  Violet Frames')
denoised_violet_frames, violetU, violetS, violetVT = denoise_svd(motion_corrected_frames_all[:,1,...],rank=200) # only denoise blue channel for now
print(f'{datetime.datetime.now().time()}: Done Denoising Violet Frames')

np.save(params['path_to_session_data']+"\denoised_blue_frames",denoised_blue_frames)
np.save(params['path_to_session_data']+"\denoised_violet_frames",denoised_violet_frames)
np.save(params['path_to_session_data']+"\blueU",blueU)
np.save(params['path_to_session_data']+"\blueS",blueS)
np.save(params['path_to_session_data']+"\blueVT",blueVT)
np.save(params['path_to_session_data']+"\violetU",violetU)
np.save(params['path_to_session_data']+"\violetS",violetS)
np.save(params['path_to_session_data']+"\violetVT",violetVT)

In [ ]:
#get_recording_paths(params['path_to_session_data'])
'''
data = recordings['recording_1'].demo_pipeline()
data = np.array(data)
np.save(params['path_to_session_data']+"\blue_frame_data",data)
'''

In [ ]:
#data = np.load(r"C:\Users\evilm\Downloads\07-18-2024\blue_frame_data.npy")

In [ ]:
#%matplotlib inline
#plt.imshow(np.mean(data,axis=0),cmap="gray")
#plt.show()